# データストアを使用した Amazon Bedrock AgentCore Gateway インターセプターによるきめ細かなアクセス制御

## 概要

このノートブックでは、**Gateway インターセプター**と権限管理用の**データストアとしての Amazon DynamoDB** を使用して、**Amazon Bedrock AgentCore Gateway** で**きめ細かなアクセス制御（FGAC）**を実施する方法を実演します。再デプロイが必要なポリシーベースのアプローチとは異なり、このパターンでは DynamoDB のレコードを更新することで**ツール権限を動的に管理**できます。

### なぜアクセス制御にデータストアを使用するのか？

従来のアプローチでは、権限を OAuth スコープや静的ポリシーに埋め込んでいました。このチュートリアルでは、より柔軟なアプローチを示します：

- **動的な権限管理**: インフラを再デプロイせずに DynamoDB でツールアクセスを更新
- **集中型ポリシーストア**: すべてのクライアント権限を一箇所で管理
- **スケーラブル**: 多くのクライアントとツール間の複雑な権限マトリックスを処理
- **監査可能**: DynamoDB の組み込み機能で権限変更を追跡

Gateway インターセプターは実行時に Amazon DynamoDB テーブルをクエリし、各クライアントがアクセスできるツールを決定し、リアルタイムのアクセス制御を提供します。

---

## このチュートリアルでカバーする内容

このチュートリアルでは、RESPONSE インターセプターを使用して **List Tools** 操作の FGAC を実装します：

📋 **FGAC を使用した List Tools（RESPONSE インターセプター）**  
   - Gateway からの tools/list レスポンスをインターセプト
   - Amazon DynamoDB をクエリしてクライアントの許可されたツールを取得（JWT からの Client ID で識別）
   - 許可されたツールのみを表示するようにツールリストをフィルタリング
   - フィルタリングされたレスポンスをクライアントに返す

**Invoke Tool** 操作での FGAC については、チュートリアル：[カスタムスコープを使用したきめ細かなアクセス制御](../01-fine-grained-access-control-using-custom-scopes.ipynb)を参照してください

![list tool](../images/FGAC_data_store.png)

---

## なぜ Gateway インターセプターを使用するのか？

Gateway インターセプターを使用すると：

- **きめ細かなアクセス制御の実装**: クライアントごと、ツールごとの認可ルールを適用
- **カスタム認可ロジックの注入**: 外部データストアをクエリして動的な権限を取得
- **監査とガバナンス**: コンプライアンスのためにツールアクセス試行をログ
- **リクエスト/レスポンスの変換**: 転送中のデータをフィルタリング、編集、または変更

インターセプターは **Gateway レイヤー** にアタッチされるため、アプリケーションコードを変更することなく、**任意の**基盤となる MCP サーバーまたはランタイムに対して集中型ポリシーを適用します。

---

## チュートリアル詳細

| 情報                     | 詳細                                                                                           |
|-------------------------|-----------------------------------------------------------------------------------------------|
| **チュートリアルタイプ**   | インタラクティブ                                                                               |
| **AgentCore コンポーネント** | Amazon Bedrock AgentCore Gateway, Gateway インターセプター                                     |
| **Gateway ターゲットタイプ** | MCP サーバー（AgentCore Runtime で実行される FastMCP）                                          |
| **インターセプタータイプ**  | AWS Lambda（RESPONSE）                                                                        |
| **インバウンド認証 IdP**   | Amazon Cognito（CUSTOM_JWT オーソライザー）                                                    |
| **データストア**          | Amazon DynamoDB（クライアントとツールの権限マッピングを保存）                                    |
| **アクセス制御**          | JWT からの Client ID + DynamoDB 権限ルックアップを使用した FGAC                                 |
| **チュートリアル構成**    | Gateway, Runtime MCP サーバー, Amazon Cognito, Gateway インターセプター, MCP ツール, Amazon DynamoDB |
| **チュートリアル業界**    | クロスバーティカル                                                                             |
| **難易度**               | 中級                                                                                           |
| **使用 SDK**             | boto3                                                                                          |

---

## 前提条件

このチュートリアルを実行するには以下が必要です：

- Jupyter notebook（Python カーネル）
- 以下の権限を持つ AWS 認証情報：
  - AWS Lambda
  - AWS IAM
  - Amazon Cognito
  - Amazon DynamoDB
  - Amazon Bedrock AgentCore サービス（コントロールプレーン + ランタイム）
- Python 3.9 以上
- AWS Lambda、IAM ロール、Amazon Cognito、Amazon Bedrock AgentCore Gateway の基本的な理解

> ⚠️ **注意:** 最後のクリーンアップセクションは、このチュートリアルで作成された AWS リソース（Gateway、Lambda、IAM ロールなど）を削除します。すべてを削除する準備ができたときのみ実行してください。

In [ ]:
# 必要なライブラリをインポート
import boto3
import json
import time
import sys
import requests
from pathlib import Path
from datetime import datetime
from botocore.exceptions import ClientError

# utils をパスに追加
current_dir = Path.cwd()
utils_dir = current_dir.parent.parent
sys.path.insert(0, str(utils_dir))

import utils

print("✓ ライブラリをインポートしました")

# このデプロイメント用の一意の識別子を生成
DEPLOYMENT_ID = datetime.now().strftime('%Y%m%d-%H%M%S')
print(f"\nデプロイメント ID: {DEPLOYMENT_ID}")

# 設定
REGION = "us-east-1"

# リソース名
USER_POOL_NAME = f"gateway-pool-{DEPLOYMENT_ID}"
DYNAMODB_TABLE_NAME = f"ClientToolPermissions-{DEPLOYMENT_ID}"
LAMBDA_FUNCTION_NAME = f"interceptor-lambda-{DEPLOYMENT_ID}"
LAMBDA_ROLE_NAME = f"interceptor-lambda-role-{DEPLOYMENT_ID}"
GATEWAY_NAME = f"interceptor-gateway-{DEPLOYMENT_ID}"

print("設定:")
print(f"  リージョン: {REGION}")
print(f"  ユーザープール: {USER_POOL_NAME}")
print(f"  DynamoDB テーブル: {DYNAMODB_TABLE_NAME}")
print(f"  Lambda 関数: {LAMBDA_FUNCTION_NAME}")
print(f"  Lambda ロール: {LAMBDA_ROLE_NAME}")
print(f"  Gateway 名: {GATEWAY_NAME}")

# AWS クライアントを初期化
cognito_client = boto3.client('cognito-idp', region_name=REGION)
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
print(f"\n✓ AWS クライアントを初期化しました")

---

## パート 1: セットアップとデプロイ

### ステップ 1.1: Amazon Cognito ユーザープールとアプリクライアントを作成

異なるアプリケーションやサービスを表す複数のアプリクライアントを作成します。

#### Client ID とアクセス制御

**Client ID** は、Amazon Cognito に登録された各アプリケーションの一意の識別子です。クライアントが認証すると、JWT トークンに Client ID が含まれ、Gateway インターセプターはこれを使用して DynamoDB で権限を検索します。

**重要なポイント:**
- すべてのクライアントは Gateway アクセス用に同じ OAuth スコープ（`gateway/tools`）を使用
- **ツールレベルの権限**は DynamoDB に保存され、Client ID を使用してクエリ
- これにより、OAuth の再構成なしで動的に権限を更新可能

**セキュリティのベストプラクティス:**

⚠️ **常に信頼できる ID プロバイダー（Amazon Cognito、Okta、Auth0、Azure AD など）からの暗号署名された JWT トークンを使用してください。** Gateway はリクエストを処理する前に署名を検証し、Client ID が本物で改ざんされていないことを確認します。

**やってはいけないこと:**
- ❌ 認証にカスタムヘッダー（例：`X-Client-ID`）を使用しない - 簡単に偽装可能
- ❌ 認証判断のためにクエリパラメータで Client ID を渡さない
- ❌ 署名されていない、または検証されていないトークンを避ける

Gateway はリクエストをインターセプターに渡す前に JWT 署名を検証し、トークン内の Client ID が本物であることを確認します。

**高度な使用法:** マルチエージェントシナリオでは、より詳細な制御のために Client ID と Agent ID を組み合わせます：`{ClientID}#{AgentID}`

In [ ]:
# 複数のアプリクライアントを持つ Cognito ユーザープールを作成
# ユーザープールを作成または取得
USER_POOL_ID = utils.get_or_create_user_pool(cognito_client, USER_POOL_NAME)

# リソースサーバーを作成または取得
RESOURCE_SERVER_ID = 'gateway'
RESOURCE_SERVER_NAME = 'Gateway Resource Server'
SCOPES = [{'ScopeName': 'tools', 'ScopeDescription': 'Access to gateway tools'}]
utils.get_or_create_resource_server(cognito_client, USER_POOL_ID, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)

# リソースサーバーの伝播を待機
time.sleep(3)

# 異なる権限レベル用に複数のアプリクライアントを作成
clients = {}
for client_name in ['full-access', 'readonly', 'calculator', 'data']:
    client_id, client_secret = utils.get_or_create_m2m_client(
        cognito_client,
        USER_POOL_ID,
        f"{client_name}-client-{DEPLOYMENT_ID}",
        RESOURCE_SERVER_ID,
        ['gateway/tools']
    )
    clients[client_name] = {'client_id': client_id, 'client_secret': client_secret}
    print(f"✓ クライアントを作成/取得: {client_name}")

# 簡単にアクセスできるように Client ID を抽出
CLIENT_ID_FULL = clients['full-access']['client_id']
CLIENT_ID_READONLY = clients['readonly']['client_id']
CLIENT_ID_CALCULATOR = clients['calculator']['client_id']
CLIENT_ID_DATA = clients['data']['client_id']

# OAuth URL を構築
POOL_DOMAIN = USER_POOL_ID.replace('_', '').lower()
DISCOVERY_URL = f"https://cognito-idp.{REGION}.amazonaws.com/{USER_POOL_ID}/.well-known/openid-configuration"
TOKEN_URL = f"https://{POOL_DOMAIN}.auth.{REGION}.amazoncognito.com/oauth2/token"

print(f"\n✓ Cognito セットアップ完了")
print(f"  ユーザープール ID: {USER_POOL_ID}")
print(f"  Discovery URL: {DISCOVERY_URL}")
print(f"  Token URL: {TOKEN_URL}")

### ステップ 1.2: Amazon DynamoDB 権限テーブルを作成

クライアントとツールの権限マッピングを保存するテーブルを作成します。各レコードは、1つのクライアントに1つのツールへのアクセスを付与します：
- **ClientID**（パーティションキー）: Amazon Cognito Client ID
- **ToolName**（ソートキー）: ツール名
- **Allowed**: ブール値フラグ

例: `ClientID: abc123, ToolName: weather_tool, Allowed: True`

このモデルは、各 Client ID が単一のアプリケーションを表すことを前提としています。ユーザーレベルの権限には、`{ClientID}#{UserID}` のような複合キーを使用します。Lambda インターセプターは、JWT からの Client ID を使用してこのテーブルをクエリします。

In [ ]:
# DynamoDB テーブルを作成
utils.create_dynamodb_table(
    table_name=DYNAMODB_TABLE_NAME,
    key_schema=[
        {'AttributeName': 'ClientID', 'KeyType': 'HASH'},
        {'AttributeName': 'ToolName', 'KeyType': 'RANGE'}
    ],
    attribute_definitions=[
        {'AttributeName': 'ClientID', 'AttributeType': 'S'},
        {'AttributeName': 'ToolName', 'AttributeType': 'S'}
    ],
    region=REGION
)

### ステップ 1.3: クライアント権限を DynamoDB にロード
各 Cognito client_id を許可されたツールにマッピングします。

In [ ]:
# クライアント権限マッピングを定義（シンプルな形式）
CLIENT_PERMISSIONS = {
    CLIENT_ID_FULL: ['weather_tool', 'database_query_tool', 'calculation_tool', 'search_tool', 'file_handler_tool'],
    CLIENT_ID_READONLY: ['weather_tool', 'search_tool'],
    CLIENT_ID_CALCULATOR: ['calculation_tool'],
    CLIENT_ID_DATA: ['database_query_tool', 'file_handler_tool', 'calculation_tool'],
}

# マッピングから権限リストを生成
SAMPLE_PERMISSIONS = [
    {'ClientID': client_id, 'ToolName': tool_name, 'Allowed': True}
    for client_id, tools in CLIENT_PERMISSIONS.items()
    for tool_name in tools
]

# 権限を DynamoDB にロード
utils.batch_write_dynamodb(
    table_name=DYNAMODB_TABLE_NAME,
    items=SAMPLE_PERMISSIONS,
    region=REGION
)

### ステップ 1.4: Lambda インターセプター用の IAM ロールを作成
Lambda に DynamoDB の読み取りと CloudWatch ログの書き込み権限を付与します。

In [ ]:
# Lambda インターセプター用の IAM ロールを作成
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
table_arn = f"arn:aws:dynamodb:{REGION}:{account_id}:table/{DYNAMODB_TABLE_NAME}"

dynamodb_policy = {
    "Effect": "Allow",
    "Action": ["dynamodb:Query", "dynamodb:GetItem"],
    "Resource": table_arn
}

LAMBDA_ROLE_ARN = utils.create_lambda_role_with_policies(
    role_name=LAMBDA_ROLE_NAME,
    policy_statements=[dynamodb_policy],
    description='DynamoDB アクセス権限を持つ Lambda インターセプターロール'
)

print(f"✓ Lambda ロール準備完了: {LAMBDA_ROLE_ARN}")


### ステップ 1.5: Lambda インターセプター関数をデプロイ
Lambda は JWT から client_id を抽出し、DynamoDB の権限に基づいてツールをフィルタリングします。

In [ ]:
# utils を使用して Lambda インターセプターをデプロイ
LAMBDA_ARN = utils.deploy_lambda_function(
    function_name=LAMBDA_FUNCTION_NAME,
    role_arn=LAMBDA_ROLE_ARN,
    lambda_code_path='src/lambda/lambda_function.py',
    environment_vars={
        'PERMISSIONS_TABLE_NAME': DYNAMODB_TABLE_NAME,
        'DYNAMODB_REGION': REGION
    },
    region=REGION
)

# Gateway に Lambda 呼び出し権限を付与
utils.grant_gateway_invoke_permission(
    function_name=LAMBDA_FUNCTION_NAME,
    region=REGION
)

print(f"\n✓ Lambda インターセプターをデプロイして設定しました: {LAMBDA_ARN}")

### ステップ 1.6: レスポンスインターセプター付きの Gateway を作成

**なぜ RESPONSE インターセプターか？**  
Gateway がすべてのターゲットからツールを収集した後、集約されたツールリストをフィルタリングします。インターセプターは JWT からの Client ID を使用して DynamoDB をクエリし、許可されたツールのみを返します。

**フロー:** リクエスト → JWT 検証 → ツール集約 → DynamoDB ルックアップ → フィルタリングされたレスポンス

In [ ]:
# Gateway IAM ロールを作成
gateway_iam_role = utils.create_agentcore_gateway_role_with_region(GATEWAY_NAME, REGION)
GATEWAY_ROLE_ARN = gateway_iam_role['Role']['Arn']

print(f"✓ Gateway ロールを作成: {GATEWAY_ROLE_ARN}")

# ロールの伝播を待機
time.sleep(10)

# Lambda インターセプター付きの Gateway を作成
print(f"\nRESPONSE インターセプター付きの Gateway を作成中...")

try:
    gateway_response = gateway_client.create_gateway(
        name=GATEWAY_NAME,
        protocolType="MCP",
        protocolConfiguration={
            "mcp": {
                "supportedVersions": ["2025-03-26"]
            }
        },
        interceptorConfigurations=[
            {
                "interceptor": {
                    "lambda": {
                        "arn": LAMBDA_ARN
                    }
                },
                "interceptionPoints": ["RESPONSE"],
                "inputConfiguration": {
                    "passRequestHeaders": True  
                }
            }
        ],
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration={
            "customJWTAuthorizer": {
                "discoveryUrl": DISCOVERY_URL,
                "allowedClients": [CLIENT_ID_FULL, CLIENT_ID_DATA, CLIENT_ID_CALCULATOR, CLIENT_ID_READONLY]
            }
        },
        roleArn=GATEWAY_ROLE_ARN
    )
    
    GATEWAY_ID = gateway_response.get('gatewayId')
    print(f"✓ Gateway を作成: {GATEWAY_ID}")
    
except Exception as e:
    print(f"\n✗ Gateway の作成に失敗: {e}")
    raise


In [ ]:
# 署名付きリクエストを使用して Gateway の準備完了を待機
print("\nGateway の準備完了を待機中...")

max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = gateway_client.get_gateway(gatewayIdentifier=GATEWAY_ID)
        status = response.get('status', 'UNKNOWN')
        
        print(f"  [{attempt + 1}/{max_attempts}] ステータス: {status}")
        
        if status == 'READY':
            GATEWAY_URL = response.get('gatewayUrl')
            print(f"\n✓ Gateway の準備が完了しました！")
            print(f"  URL: {GATEWAY_URL}")
            print(f"  インターセプター: RESPONSE (Lambda: {LAMBDA_ARN})")
            break
        elif status == 'FAILED':
            print(f"\n✗ Gateway の作成に失敗しました")
            raise Exception("Gateway failed")
    except Exception as e:
        print(f"  [{attempt + 1}/{max_attempts}] エラー: {e}")
        raise
    
    time.sleep(10)
else:
    print(f"\n⚠ Gateway の待機がタイムアウトしました")
    raise Exception("Gateway timeout")

### ステップ 1.7: サンプルツールを Gateway に登録
ツール Lambda をデプロイし、Gateway ターゲットとして登録します。

In [ ]:
# ツールモジュールをインポート
import zipfile
import io
sys.path.insert(0, str(Path.cwd()))

from src.tools import weather_tool, database_query_tool, calculation_tool, search_tool, file_handler_tool

# ツール Lambda 用の IAM ロールを作成
TOOL_ROLE_ARN = utils.create_lambda_role(
    role_name=f"tool-lambda-role-{DEPLOYMENT_ID}",
    description='ツール Lambda 関数用ロール'
)

# ツール Lambda 関数をインポートしてデプロイ
print("ツール Lambda 関数をデプロイ中...")
sys.path.insert(0, str(Path.cwd()))
from src.tools import weather_tool, database_query_tool, calculation_tool, search_tool, file_handler_tool

lambda_client = boto3.client('lambda', region_name=REGION)

tools_to_deploy = [
    ('weather_tool', weather_tool),
    ('database_query_tool', database_query_tool),
    ('calculation_tool', calculation_tool),
    ('search_tool', search_tool),
    ('file_handler_tool', file_handler_tool),
]

deployed_tools = []

for tool_name, tool_module in tools_to_deploy:
    print(f"  {tool_name} をデプロイ中...")
    
    function_name = f"{tool_name.replace('_', '-')}-{DEPLOYMENT_ID}"
    tool_code_path = Path(tool_module.__file__)
    
    lambda_arn = utils.deploy_lambda_function(
        function_name=function_name,
        role_arn=TOOL_ROLE_ARN,
        lambda_code_path=str(tool_code_path),
        environment_vars={'TOOL_NAME': tool_name},
        description=f'{tool_name} 関数',
        region=REGION
    )
    
    tool_definition = getattr(tool_module, 'TOOL_DEFINITION', {
        "name": tool_name,
        "description": f"{tool_name} 関数"
    })
    
    deployed_tools.append({
        'tool_name': tool_name,
        'function_name': function_name,
        'lambda_arn': lambda_arn,
        'tool_definition': tool_definition
    })

print(f"✓ {len(deployed_tools)} 個のツール Lambda をデプロイしました")

In [ ]:
# ツールを Gateway ターゲットとして登録
print("ツールを Gateway ターゲットとして登録中...")
created_targets = []

for tool in deployed_tools:
    print(f"  {tool['tool_name']} を登録中...")
    
    try:
        response = gateway_client.create_gateway_target(
            gatewayIdentifier=GATEWAY_ID,
            name=f"{tool['tool_name'].replace('_', '-')}-target",
            targetConfiguration={
                "mcp": {
                    "lambda": {
                        "lambdaArn": tool["lambda_arn"],
                        "toolSchema": {"inlinePayload": [tool["tool_definition"]]}
                    }
                }
            },
            credentialProviderConfigurations=[{
                "credentialProviderType": "GATEWAY_IAM_ROLE"
            }]
        )
        
        target_id = response['targetId']
        print(f"    ✓ ターゲットを作成: {target_id}")
        
        # ターゲットが READY になるまで待機
        for attempt in range(18):
            status_response = gateway_client.get_gateway_target(
                gatewayIdentifier=GATEWAY_ID,
                targetId=target_id
            )
            status = status_response.get('status')
            
            if status == 'READY':
                print(f"    ✓ ターゲットは READY です")
                created_targets.append({
                    'tool_name': tool['tool_name'],
                    'target_id': target_id,
                    'lambda_arn': tool['lambda_arn']
                })
                break
            elif status == 'FAILED':
                print(f"    ✗ ターゲットが FAILED になりました")
                break
            
            time.sleep(10)
            
    except Exception as e:
        print(f"    ✗ 失敗: {e}")

print(f"✓ {len(created_targets)}/{len(deployed_tools)} 個のターゲットを登録しました")

# クリーンアップ用に保存
DEPLOYED_TOOL_FUNCTIONS = [t['function_name'] for t in deployed_tools]
CREATED_TARGET_IDS = [t['target_id'] for t in created_targets]

---

## パート 2: テスト

### ステップ 2.1: 異なる Client ID でテスト
各クライアントが許可されたツールのみを参照できることを確認します。

In [ ]:
# 期待される権限を持つテストクライアントを定義
print("="*80)
print("異なるクライアントできめ細かなアクセス制御をテスト")
print("="*80)

test_clients = [
    {
        'name': 'full-access',
        'client_id': CLIENT_ID_FULL,
        'expected_tools': ['weather_tool', 'database_query_tool', 'calculation_tool', 'search_tool', 'file_handler_tool']
    },
    {
        'name': 'readonly',
        'client_id': CLIENT_ID_READONLY,
        'expected_tools': ['weather_tool', 'search_tool']
    },
    {
        'name': 'calculator',
        'client_id': CLIENT_ID_CALCULATOR,
        'expected_tools': ['calculation_tool']
    },
    {
        'name': 'data',
        'client_id': CLIENT_ID_DATA,
        'expected_tools': ['database_query_tool', 'file_handler_tool', 'calculation_tool']
    }
]

print(f"\n✓ {len(test_clients)} 個のテストクライアントを設定しました")

In [ ]:
# utils を使用して Cognito からクライアントシークレットを取得
client_secrets = utils.get_client_secrets(
    cognito_client=cognito_client,
    user_pool_id=USER_POOL_ID,
    client_configs=test_clients
)

In [ ]:
# 各クライアントのツールへのアクセスをテスト
test_results = []

for client_config in test_clients:
    print(f"\n{'='*60}")
    print(f"テストクライアント: {client_config['name']}")
    print(f"{'='*60}")
    print(f"  Client ID: {client_config['client_id']}")
    print(f"  期待されるツール: {client_config['expected_tools']}")
    
    client_id = client_config['client_id']
    client_secret = client_secrets.get(client_id)
    
    if not client_secret:
        print(f"  ✗ クライアントシークレットがありません、スキップします")
        test_results.append({'name': client_config['name'], 'passed': False, 'reason': 'シークレットなし'})
        continue
    
    try:
        # ステップ 1: TOKEN_URL を使用してアクセストークンを取得
        print("\n  ステップ 1: アクセストークンをリクエスト中...")
        print(f"  Token URL: {TOKEN_URL}")
        time.sleep(2)  # 短い待機
        
        token_data = utils.get_token(
        user_pool_id=USER_POOL_ID,
        client_id=client_id,
        client_secret=client_secret,
        scope_string="gateway/tools",
        REGION=REGION
    )

        if 'error' in token_data:
            print(f"    ✗ トークンリクエスト失敗: {token_data['error']}")
            test_results.append({'name': client_config['name'], 'passed': False, 'reason': 'トークン失敗'})
            continue

        token = token_data['access_token']
        print(f"    ✓ トークンを取得 ({token_data.get('expires_in')}秒で期限切れ)")
        
        # ステップ 2: MCP プロトコルを使用して Gateway を呼び出しツールを一覧表示
        print("\n  ステップ 2: Gateway を呼び出してツールを一覧表示中...")
        
        # MCP tools/list リクエスト
        mcp_request = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "tools/list",
            "params": {}
        }
        
        response = requests.post(
            GATEWAY_URL,
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            json=mcp_request
        )
        
        if response.status_code != 200:
            print(f"    ✗ Gateway リクエスト失敗: {response.status_code}")
            print(f"    レスポンス: {response.text}")
            test_results.append({'name': client_config['name'], 'passed': False, 'reason': f'HTTP {response.status_code}'})
            continue
        
        result = response.json()
        
        if 'error' in result:
            print(f"    ✗ MCP エラー: {result['error']}")
            test_results.append({'name': client_config['name'], 'passed': False, 'reason': 'MCP エラー'})
            continue
        
        # ツール名を抽出
        tools = result.get('result', {}).get('tools', [])
        actual_tool_names = [
            tool['name'].split('___')[1] if '___' in tool['name'] else tool['name']
            for tool in tools
        ]
        
        print(f"    ✓ {len(actual_tool_names)} 個のツールを受信")
        print(f"    解析された名前: {actual_tool_names}")
        
        # ステップ 3: 権限を検証
        print("\n  ステップ 3: 権限を検証中...")
        
        expected_tools = set(client_config['expected_tools'])
        actual_tools = set(actual_tool_names)
        
        print(f"    期待: {sorted(expected_tools)}")
        print(f"    実際:   {sorted(actual_tools)}")
        
        if expected_tools == actual_tools:
            print(f"\n  ✅ 合格: クライアントは正しい権限を持っています")
            test_results.append({'name': client_config['name'], 'passed': True})
        else:
            print(f"\n  ❌ 失敗: 権限の不一致")
            
            missing = expected_tools - actual_tools
            if missing:
                print(f"    不足しているツール: {sorted(missing)}")
            
            extra = actual_tools - expected_tools
            if extra:
                print(f"    余分なツール: {sorted(extra)}")
            
            test_results.append({'name': client_config['name'], 'passed': False, 'reason': '不一致'})
    
    except Exception as e:
        print(f"\n  ✗ テストが例外で失敗: {e}")
        import traceback
        traceback.print_exc()
        test_results.append({'name': client_config['name'], 'passed': False, 'reason': str(e)})

In [ ]:
# テストサマリーを表示
print(f"\n{'='*80}")
print("テストサマリー")
print(f"{'='*80}")

passed_count = sum(1 for r in test_results if r['passed'])
total_count = len(test_results)

for result in test_results:
    status = "✅ 合格" if result['passed'] else "❌ 失敗"
    reason = f" ({result.get('reason', '')})" if not result['passed'] and 'reason' in result else ""
    print(f"  {status}: {result['name']}{reason}")

print(f"\n合計: {passed_count}/{total_count} 合格")

if passed_count == total_count:
    print("\n🎉 すべてのテストに合格しました！きめ細かなアクセス制御が正常に動作しています。")
else:
    print("\n⚠️ 一部のテストが失敗しました。上記のログで詳細を確認してください。")

---

## パート 3: クリーンアップ

⚠️ **警告: これにより、パート 1 で作成されたすべてのリソースが削除されます！**

すべてをクリーンアップする場合のみ、このセクションを実行してください。

### ステップ 3.1: 作成されたリソースを削除

In [ ]:
# クリーンアップ - 作成されたすべてのリソースを削除
print("クリーンアップを開始...")

# 1. Gateway ターゲットを削除
if 'CREATED_TARGET_IDS' in globals() and 'GATEWAY_ID' in globals():
    utils.delete_gateway_targets(gateway_client, GATEWAY_ID, CREATED_TARGET_IDS)

# 2. Gateway を削除
if 'GATEWAY_ID' in globals():
    utils.delete_gateway(gateway_client, GATEWAY_ID)
    print("✓ Gateway を削除しました")

# 3. Lambda 関数を削除（ツール + インターセプター）
lambda_functions_to_delete = []
if 'DEPLOYED_TOOL_FUNCTIONS' in globals():
    lambda_functions_to_delete.extend(DEPLOYED_TOOL_FUNCTIONS)
if 'LAMBDA_FUNCTION_NAME' in globals():
    lambda_functions_to_delete.append(LAMBDA_FUNCTION_NAME)

if lambda_functions_to_delete:
    utils.delete_lambda_functions(lambda_functions_to_delete, REGION)

# 4. IAM ロールを削除
if 'LAMBDA_ROLE_NAME' in globals():
    utils.delete_iam_role(LAMBDA_ROLE_NAME)
if 'TOOL_ROLE_NAME' in globals():
    utils.delete_iam_role(f"tool-lambda-role-{DEPLOYMENT_ID}")
if 'GATEWAY_ROLE_ARN' in globals():
    utils.delete_iam_role(f"gateway-role-{DEPLOYMENT_ID}")

# 5. DynamoDB テーブルを削除
if 'DYNAMODB_TABLE_NAME' in globals():
    utils.delete_dynamodb_table(DYNAMODB_TABLE_NAME, REGION)

# 6. Cognito ユーザープールを削除
if 'USER_POOL_ID' in globals():
    utils.delete_cognito_user_pool(USER_POOL_ID, REGION)

print("\n✓ クリーンアップ完了!")

---

# まとめ

このノートブックで完全なライフサイクルを完了しました：

1. ✅ **セットアップ** - DynamoDB、Lambda、IAM ロール、Gateway を作成
2. ✅ **テスト** - 実際の Gateway を通じてツールフィルタリングを検証
3. ✅ **クリーンアップ** - すべてのリソースを削除

## 実演した内容

- DynamoDB 権限を使用した**エージェントベースのツールフィルタリング**
- Gateway レスポンスを変更する **Lambda RESPONSE インターセプター**
- リクエストチェーンを通じた**カスタムヘッダー伝播**（Agent-ID）
- **完全なリソースライフサイクル**管理

## 次のステップ

- 異なる構成で再度実行
- より多くのカスタムエージェントとツールを追加
- 実際の AgentCore Runtime エージェントと統合
- デバッグのために CloudWatch ログを監視